## Install libraries

In [1]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [2]:
!pip install langchain bitsandbytes accelerate langchain_community sentence-transformers faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_

In [3]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 6.5 MB/s eta 0:00:00


## Dataset preparation & preprocessing

### 1-Import libraries

In [4]:
from langchain.document_loaders import PyPDFLoader  # Imports the PyPDFLoader class for loading PDF documents.
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Imports a text splitter that can recursively split text based on character count.
from langchain.embeddings import HuggingFaceEmbeddings  # Imports embedding functionality that uses models from HuggingFace.
from langchain.prompts import ChatPromptTemplate  # Imports the ChatPromptTemplate class for generating chat prompts.
from langchain import HuggingFaceHub  # Imports HuggingFaceHub, a module for interacting with the Hugging Face Model Hub.
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline  # Imports HuggingFacePipeline for using Hugging Face transformers in a pipeline.
from transformers import AutoModelForCausalLM, AutoTokenizer  # Imports classes for automatically loading causal language models and their tokenizers.
from langchain.chains import RetrievalQA  # Imports the RetrievalQA class, used for setting up a retrieval-based question-answering system.
from langchain_community.vectorstores import FAISS
from transformers import BitsAndBytesConfig
# Use a pipeline as a high-level helper
from transformers import pipeline
from langchain.chains import RetrievalQA

import torch  # Imports the PyTorch library for handling tensors and deep learning operations.
import os  # Imports the os module, which provides functions for interacting with the operating system.
import warnings  # Imports the warnings module, which is used to manage warnings.
import faiss  # Import the FAISS library

warnings.filterwarnings('ignore')  # Sets up the warnings filter to ignore all warnings.


### 2- load the PDF

In [5]:
pdf_file = PyPDFLoader("/content/Attention_is_all_you_need.pdf")  # Creates an instance of PyPDFLoader for a specific PDF file.
pages = pdf_file.load_and_split()  # Loads the PDF file and splits its content into pages, storing them in 'pages'.

In [6]:
len(pages)

16

###3- Split our PDF to chunks

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1024,  # Sets the size of text chunks to 1024 characters.
    chunk_overlap = 64,  # Sets the overlap between consecutive chunks to 64 characters.
)

In [8]:
chunks = text_splitter.split_documents(pages)
len(chunks)

48

In [9]:
chunks[23]

Document(page_content='recurrent layers, by a factor of k. Separable convolutions [ 6], however, decrease the complexity\nconsiderably, to O(k·n·d+n·d2). Even with k=n, however, the complexity of a separable\nconvolution is equal to the combination of a self-attention layer and a point-wise feed-forward layer,\nthe approach we take in our model.\nAs side benefit, self-attention could yield more interpretable models. We inspect attention distributions\nfrom our models and present and discuss examples in the appendix. Not only do individual attention\nheads clearly learn to perform different tasks, many appear to exhibit behavior related to the syntactic\nand semantic structure of the sentences.\n5 Training\nThis section describes the training regime for our models.\n5.1 Training Data and Batching\nWe trained on the standard WMT 2014 English-German dataset consisting of about 4.5 million\nsentence pairs. Sentences were encoded using byte-pair encoding [ 3], which has a shared source-', m

###4- Load the Embading model

In [10]:
Embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",  # Specifies the embading model to use from Hugging Face's model hub.
    model_kwargs={"device": "cuda"}  # sets model_kwargs as a dictionary specifying the device as CUDA for GPU acceleration.
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### 5- Create a vector database

Now attempts to create a vector database (index) using FAISS (Facebook AI Similarity Search), which is a library that efficiently handles large-scale clustering and similarity searches among vectors.

In [11]:
vector_db = FAISS.from_texts([str(chunk)for chunk in chunks], Embeddings)

###6- Test the RAG with no LLM (no generative content)

In [12]:
question = """
what is the attention function?
"""

relevent_results = vector_db.similarity_search(question, k=2)
print(relevent_results[1])

page_content="page_content='recurrent layers, by a factor of k. Separable convolutions [ 6], however, decrease the complexity\\nconsiderably, to O(k·n·d+n·d2). Even with k=n, however, the complexity of a separable\\nconvolution is equal to the combination of a self-attention layer and a point-wise feed-forward layer,\\nthe approach we take in our model.\\nAs side benefit, self-attention could yield more interpretable models. We inspect attention distributions\\nfrom our models and present and discuss examples in the appendix. Not only do individual attention\\nheads clearly learn to perform different tasks, many appear to exhibit behavior related to the syntactic\\nand semantic structure of the sentences.\\n5 Training\\nThis section describes the training regime for our models.\\n5.1 Training Data and Batching\\nWe trained on the standard WMT 2014 English-German dataset consisting of about 4.5 million\\nsentence pairs. Sentences were encoded using byte-pair encoding [ 3], which has a s

###7- Prompt & LLM Integration

In [13]:
prompt = """
Using this piece of information:
\n
{context}
\n
Answer the following question:
\n
{question}
\n
Answer:
\n
"""

prompt = ChatPromptTemplate.from_template(prompt)

In [14]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4"
)

In [15]:
from huggingface_hub import notebook_login
notebook_login()

In [16]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", quantization_config= bnb_config)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [1]:
pipe = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.2", tokenizer = tokenizer, max_new_tokens = 64)
lc_pipeline = HuggingFacePipeline(pipeline = pipe)

NameError: name 'pipeline' is not defined

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm = lc_pipeline,
    retriever = vector_db.as_retriever(search_kward = {'k':3} ),
    return_source_documents = True,
    chain_type_kwargd = {'prompt:' prompt}
)

Test the RAG with LLM

In [ ]:
question = """
what is the attention function?
"""
result = qa_chain({"query": question})

NameError: name 'qa_chain' is not defined

In [ ]:
print(result['result'].split("Answer:")[1])

In [ ]:
print(result["source_documents"])